## Frobenius action on the relative de Rham cohomology of Elliptic Curves

The following notebook contains the necessary steps to compute the Frobenius action on the relative $H_{dR}$

In [30]:
p = 5
Z5 = Zp(p, prec = 10, type = 'capped-abs', print_mode = 'series')
P.<t> = PolynomialRing(QQ)
P = FractionField(P)
R.<x> = PowerSeriesRing(P)
S.<z> = PowerSeriesRing(R)
S

Power Series Ring in z over Power Series Ring in x over Fraction Field of Univariate Polynomial Ring in t over Rational Field

In [62]:
f = x*(x-1)*(x-3)
frob_x = x^p
#frob_y = y^p*(1+(f(x^p)-f(x)^p)*1/f(x)^p)^(1/2)
frob_z = z^p*(1+(f(x^p)-f(x)^p)*z^(2*p))^(-1/2)
frob_z

z^5 + (120*x^5 - 810*x^6 + 4725/2*x^7 - 3960*x^8 + 4215*x^9 - 2970*x^10 + 1405*x^11 - 440*x^12 + 175/2*x^13 - 10*x^14)*z^15 + O(z^25)

We now consider the following differentials
$$ \omega = \frac{dx}{y}=zdx$$ 
$$\eta = \frac{xdx}{y}=zxdx$$
and the action of the Frobenius morphism $\varphi$
$$ \varphi^*(\omega) = px^{p-1}\varphi(z)dx$$
$$ \varphi^*(\eta) = px^{2p-1}\varphi(z)dx$$

This is an andomorphism of the de Rham cohomology group $H_{dR}(E/\mathbb{Q}_5)$. In particular there exists a matrix $M$ and two rational functions $f_\omega,f_\eta$ such that the following holds
$$ \begin{pmatrix}\varphi^*(\omega) \\ \varphi^*(\eta)\end{pmatrix} = M\cdot \begin{pmatrix}\omega \\ \eta \end{pmatrix} + \begin{pmatrix} df_\omega \\ df_\eta\end{pmatrix}$$

In [50]:
frob_omega = p*x^(p-1)*frob_z
frob_eta = p*x^(2*p-1)*frob_z

The following is a function to reduce the coefficient of the previous power series in the form
$$ \sum_{i\in \mathbb{Z}}c_{i}z^{pi}$$
with $c_i$ polynomials in $x$ of degree at most $2g+1$

In [51]:
V = LaurentSeriesRing(S)
def reduction_deg_2g(f,pol):
    """
    Given a polynomial in x, returns the representative in K[x,y,z]/(y^2-f(x),yz-1) that is a polynomial in K[x][z,z^(-1)] with the 
    degree in x <=2
    """
    if pol(z=1).degree()<3:
        return pol
    else: 
        n = pol(z=1).degree()
        pol += - pol(z=1).coefficients()[-1]*(f^(floor(n/3))-1/z^(2*floor(n/3)))*x^(n%3)
        return reduction_deg_2g(f,pol)

Test to check if algorithm for reduction of monomials work

In [52]:
V.<s,r,t> = PolynomialRing(QQ)
I = V.ideal(r^2-1-26*s-s^3,r*t-1)
I.reduce(s^22)

s*r^14 - 182*s^2*r^12 - 7*s*r^12 + 1092*s^2*r^10 + 14196*r^12 - 984235*s*r^10 + 31985590*s^2*r^8 - 85176*r^10 + 4921245*s*r^8 - 127949640*s^2*r^6 - 665144116*r^8 + 25939082659*s*r^6 - 288953239146*s^2*r^4 + 2661144304*r^6 - 77836933013*s*r^4 + 578162380484*s^2*r^2 + 1875451651788*r^4 - 54217194935481*s*r^2 + 140877982475178*s^2 - 3756225819320*r^2 + 54269088848831*s + 1878778238324

The following function takes a power series in $z$ and returns the corresponding power series where the coefficients are the $c_i$ above

In [53]:
def reduction_coeff(pow_ser):
    """
    Given a power series in z with coefficients polynomials in x, uses the function reduction_deg_2g() to return the power series
    with coefficients polynomials in x fo degree <3 in the polynomial ring K[x,y,z]/(y^2-f(x),yz-1)
    """
    pow_ser_2 = pow_ser
    for i in range(len(pow_ser_2.coefficients())):
        pow_ser += -z^pow_ser_2.exponents()[i]*pow_ser_2.coefficients()[i]+z^pow_ser_2.exponents()[i]*reduction_deg_2g(f,pow_ser_2.coefficients()[i])
    return pow_ser


In [54]:
red_frob_eta = reduction_coeff(frob_eta)
red_frob_omega = reduction_coeff(frob_omega)
red_frob_omega

(-30 + 5*x)*z^3 + (-65075/2 - 4610*x - 1395/2*x^2)*z^5 + (-1811600 - 598075/2*x - 197600*x^2)*z^7 + (-25298850 - 1432750*x - 15756575/2*x^2)*z^9 + (-115405250 + 80736875/2*x - 91455350*x^2)*z^11 + (-158776200 + 293290350*x - 745356925/2*x^2)*z^13 + (476328600*x - 476328600*x^2)*z^15 + O(z^25)

Using the defining equation
$$ \frac{1}{z^2}=y^2=f(x)$$
we obtain the relation at the level of differentials given by
$$ dz = -\frac{1}{2}z^3f'(x).$$
From this we deduce
$$ d(x^iz^j) = \left(ix^{i-1}z^j-\frac{1}{2}jf'(x)x^iz^{j+2}\right)dx$$
The following function computes $d(x^iz^j)$ in the reduced form.

In [55]:
def differential_zn(f,i,n):
    """
    Compute the exact differential d(x^iz^n) with i=0,1,2 and n integer
    return p(x,z) where p(x,z)dx=d(x^i z^n) and p(x,z)=\sum p_i(x)z^i with degree(p_i(x))<3 
    """
    if n == 0:
        return i*x^(i-1)*z^0
    return (i*x^(i-1)*z^n+reduction_deg_2g(f,-1/2*n*f.derivative(x)*x^i)*z^(n+2))

In order to obtain the image of $\varphi^*(\omega)$ and $\varphi^*(\eta)$ in $H_{dR}$ we will need to understand what are the exact form $f_\omega, f_\eta$ for which
$$ \begin{pmatrix}\varphi^*(\omega) \\ \varphi^*(\eta)\end{pmatrix} = M\cdot \begin{pmatrix}\omega \\ \eta \end{pmatrix} + \begin{pmatrix} df_\omega \\ df_\eta\end{pmatrix}.$$
This computation reduces to a linear algebraic relation of the highest (for the positive part) and lowest (for the negative one) power of $z$ appearing in the expression $d(x^iz^j)$ previously computed.

In [56]:
def matrix_coeff_differentials_pos(n):
    matrix_diff = matrix(P,3)
    for i in range(3):
        if differential_zn(f,i,n) == 0:
            pol_coef = 0
        else:
            pol_coef = differential_zn(f,i,n).coefficients()[-1]
        for j in range(3):
            if pol_coef == 0:
                matrix_diff[j,i] = 0
            else:
                matrix_diff[j,i] = pol_coef[j]
    return matrix_diff

def matrix_coeff_differentials_neg(n):
    matrix_diff = matrix(P,3)
    for i in range(3):
        if differential_zn(f,i,n) == 0:
            pol_coef = 0
        else:
            if i == 0:
                pol_coef = differential_zn(f,i,n-2).coefficients()[0]
            else:
                pol_coef = differential_zn(f,i,n).coefficients()[0]
        for j in range(3):
            if pol_coef == 0:
                matrix_diff[j,i] = 0
            else:
                matrix_diff[j,i] = pol_coef[j]
    return matrix_diff

def vector_coeff(pol):
    return vector([pol[0],pol[1],pol[2]])

In [57]:
def reduction_z_pos(pow_ser,exact_form=0):
    if pow_ser.degree()<3:
        return (pow_ser,exact_form)
    else:
        k = pow_ser.degree()
        M = matrix_coeff_differentials_pos(k-2)
        V = vector_coeff(pow_ser.coefficients()[-1])
        X = M.solve_right(V)
        exact_form += X.dot_product(vector([x^0*z^(k-2),x^1*z^(k-2),x^2*z^(k-2)]))
        return reduction_z_pos(pow_ser-X.dot_product(vector([differential_zn(f,0,k-2),differential_zn(f,1,k-2),differential_zn(f,2,k-2)])),exact_form)

In [77]:
def reduction_z_neg(pow_ser,exact_form=0):
    if pow_ser.exponents()[0]>0:
        return (pow_ser,exact_form)
    else:
        k = pow_ser.exponents()[0]
        M = matrix_coeff_differentials_neg(k)
        V = vector_coeff(pow_ser.coefficients()[0])
        X = M.solve_right(V)
        exact_form += X.dot_product(vector([x^0*z^(k-2),x^1*z^k,x^2*z^k]))
        return reduction_z_neg(pow_ser-X.dot_product(vector([differential_zn(f,0,k-2),differential_zn(f,1,k),differential_zn(f,2,k)])),exact_form)

def reduction_z(pow_ser):
    red_pow_ser_pos, exact_pos = reduction_z_pos(pow_ser)
    red_pow_ser_neg, exact_neg = reduction_z_neg(red_pow_ser_pos,exact_pos)

    exact_form = exact_neg + red_pow_ser_neg[1][2]/(differential_zn(f,0,-1)[1][2])*z^(-1)
    red_pow_ser_fin = red_pow_ser_neg -red_pow_ser_neg[1][2]/(differential_zn(f,0,-1)[1][2])*differential_zn(f,0,-1)

    return (red_pow_ser_fin,exact_form)
    

In [79]:
red_frob_eta_fin, exact_form_omega= reduction_z(red_frob_eta)
red_frob_omega_fin, exact_form_eta= reduction_z(red_frob_eta)


In [80]:
matrix_frobenius = matrix(P,2)
matrix_frobenius[0,0] = red_frob_omega_fin[1][0]
matrix_frobenius[0,1] = red_frob_omega_fin[1][1]
matrix_frobenius[1,0] = red_frob_eta_fin[1][0]
matrix_frobenius[1,1] = red_frob_eta_fin[1][1]

In [81]:
matrix_frobenius

[   4232259/1456 -45206039/11648]
[   4232259/1456 -45206039/11648]

In [18]:
exact_form_omega
exact_form_eta

(O(5^10) + (2 + O(5^9))*x)*z^-1 + (2*5 + 3*5^2 + 4*5^3 + 2*5^4 + 2*5^6 + 5^8 + 2*5^9 + O(5^10) + (2*5 + 4*5^2 + 4*5^3 + 2*5^4 + 4*5^5 + 2*5^6 + 5^7 + 3*5^8 + 5^9 + O(5^10))*x + (2*5^2 + 3*5^3 + 5^4 + 3*5^5 + 5^7 + 3*5^8 + 3*5^9 + O(5^10))*x^2)*z + (3*5 + 2*5^2 + 2*5^3 + 4*5^4 + 3*5^5 + 2*5^6 + 5^7 + 5^9 + O(5^10) + (3*5 + 4*5^3 + 5^4 + 2*5^5 + 2*5^6 + 5^7 + 4*5^9 + O(5^10))*x + (3*5 + 2*5^2 + 3*5^3 + 3*5^4 + 5^5 + 2*5^6 + 2*5^7 + 3*5^8 + O(5^10))*x^2)*z^3 + (3*5^4 + 2*5^5 + 3*5^6 + 3*5^7 + 4*5^9 + O(5^10) + (3*5^2 + 3*5^3 + 4*5^5 + 4*5^6 + 5^8 + 2*5^9 + O(5^10))*x + (3*5 + 2*5^2 + 4*5^4 + 4*5^6 + 4*5^8 + O(5^10))*x^2)*z^5 + (2*5^2 + 2*5^3 + 2*5^4 + 3*5^5 + 4*5^6 + 4*5^7 + 4*5^8 + 5^9 + 2*5^10 + O(5^11) + (2*5^2 + 3*5^3 + 5^4 + 3*5^5 + 5^6 + 4*5^8 + 5^9 + 3*5^10 + O(5^11))*x + (3*5^3 + 2*5^4 + 2*5^5 + 5^6 + 4*5^7 + 2*5^8 + 3*5^9 + 5^10 + O(5^11))*x^2)*z^7 + (3*5^3 + 3*5^5 + 3*5^6 + 4*5^7 + 4*5^8 + 3*5^9 + 5^10 + O(5^11) + (3*5^4 + 5^5 + 5^6 + 5^7 + 2*5^8 + 5^9 + 4*5^10 + O(5^11))*x + (3

In [82]:
def matrix_frobenius(p,frob_omega,frob_eta,prec=10, exact_dif = False):

    Z5 = Zp(p, prec, type = 'capped-abs', print_mode = 'series')
    
    red_frob_eta = reduction_coeff(frob_eta)
    red_frob_omega = reduction_coeff(frob_omega)

    red_frob_eta_fin, exact_form_omega= reduction_z(red_frob_eta)
    red_frob_omega_fin, exact_form_eta= reduction_z(red_frob_eta)

    matrix_frobenius = matrix(Z5,2)
    matrix_frobenius[0,0] = red_frob_omega_fin[1][0]
    matrix_frobenius[0,1] = red_frob_omega_fin[1][1]
    matrix_frobenius[1,0] = red_frob_eta_fin[1][0]
    matrix_frobenius[1,1] = red_frob_eta_fin[1][1]

    if exact_dif:
        return (matrix_frobenius, [exact_form_omega,exact_form_eta])
    else:
        return matrix_frobenius
    